### 初赛和复赛过程中，更换了好几次数据，这里以复赛最后一次更换的数据为准

**注：原始数据其实是有不少缺失值的，这些缺失值均用-1来表示，初赛时候尝试填充这些缺失值，但是发现性能并没有提升多少，所以在复赛时候就没有对缺失值进行填充了**

In [ ]:
import time
import datetime
import gc
from collections import Counter

import pandas as pd
import numpy as np

In [ ]:
# 读入训练和预测集
train = pd.read_csv('../Data/round2_train.txt', sep='\s+')
test = pd.read_csv('../Data/round2_ijcai_18_test_b_20180510.txt', sep='\s+')

In [ ]:
# 时间转化
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

def convert_data_time(df):
    df['time'] = df.context_timestamp.apply(timestamp_datetime)
    df['day'] = df.time.apply(lambda x: int(x[8:10]))
    df['hour'] = df.time.apply(lambda x: int(x[11:13]))
    return df

In [ ]:
train = convert_data_time(train)
test = convert_data_time(test)

In [ ]:
# 查看训练集的预测集含有哪些天以及每天有多少个样本
Counter(train['day'])
Counter(test['day'])

In [ ]:
# 已知训练集样本的日期是上月的31号到本月的7号，且预测集样本的日期均为7号
# 计算训练集每天的商品转化率
days = [31, 1, 2, 3, 4, 5, 6, 7]
conv_rates = []

for day in days:
    conv_rate = train[train['day'] == day]['is_trade'].sum() / len(train[train['day'] == day])
    conv_rates.append(conv_rate)

In [ ]:
# 可见7号转化率猛增,5、6号转化率下降，其余天数转化率较为稳定；
# 由此可知7号是较为特殊的日子，结合其转化率猛增的特性，初步判断为双十一这样的日子(日期脱敏，数据集里的日期不代表实际日期)
conv_rates 

In [ ]:
# 7号的训练集样本单独拿出来考虑
train_day_7 = train[train['day'] == 7]

In [ ]:
# 5号和6号的训练集样本单独拿出来考虑
train_day_56 = train[(train['day'] > 4) & (train['day'] < 7)]

In [ ]:
# 统计5号和6号用户看了哪些商品
item_56 = Counter(train_day_56['item_id'])

In [ ]:
# 统计7号用户看了哪些商品
item_7 = Counter(train_day_7['item_id'])

In [ ]:
# 7号出现的商品有多少在5、6两天出现过
# 可见是0.89左右，占比挺大
count = 0
for item in item_7.keys():
    if item in item_56.keys():
        count += 1
print(count / len(item_7))

In [ ]:
# 统计7号成交的有哪些商品
item_7_trade = Counter(train_day_7[train_day_7['is_trade'] == 1]['item_id'])

In [ ]:
# 7号成交的商品有多少在5、6两天出现过
# 可见是0.97左右，占比非常大
# 符合日常认知，即双十一这天要买的商品，一般在前两天会选好，然后在双十一这天下单
# 也可以解释为什么5号，6号转化率商品转化率下降，毕竟双十一会打折
count = 0
for item in item_7_trade.keys():
    if item in item_56.keys():
        count += 1
print(count / len(item_7_trade))

In [ ]:
# 上面对商品的分析同样可以用来分析用户
# 分别统计5,6,7三天出现的用户
user_56 = Counter(train_day_56['user_id'])
user_7 = Counter(train_day_7['user_id'])

In [ ]:
# 统计7号出现的用户有多少在5和6号出现过
# 占比不大，说明有些用户在双十一前两天并没有浏览，而选择在双十一当天做活动的时候浏览商品
count = 0
for user in user_7.keys():
    if user in user_56.keys():
        count += 1
print(count / len(user_7))

In [ ]:
# 发生购买行为的用户统计
user_7_trade = Counter(train_day_7[train_day_7['is_trade'] == 1]['user_id'])

In [ ]:
# 计算这些用户出现在双十一前两天的比例
count = 0
for user in user_7_trade.keys():
    if user in user_56.keys():
        count += 1
print(count / len(user_7_trade))

### 总结
- 训练集和预测集给出了31号和1到7号这8天的数据样本，其中7号数据的转化率明显大于前几天的数据；
- 考虑到7号的转化率和前面日期的转化率有所不同，同时由于自身机器内存不足，只考虑训练集中的7号数据来作为训练集；
- 划分数据见 **Step\_2\_复赛数据划分**